<a href="https://colab.research.google.com/github/EdmilsonSantana/tcc-2022-2/blob/main/notebooks/PTT5_Fine_Tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Instalação de pacotes

In [1]:
%%bash
pip install nltk
pip install datasets
pip install transformers[torch]
pip install tokenizers
pip install evaluate
pip install rouge_score
pip install sentencepiece
pip install huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 17.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-non

In [2]:
import evaluate
import nltk
import json
import numpy as np
from datasets import Dataset
from transformers import T5Tokenizer
from transformers import T5ForConditionalGeneration
from transformers import Seq2SeqTrainingArguments
from transformers import Seq2SeqTrainer
from transformers import EvalPrediction
from transformers import DataCollatorForSeq2Seq
import gc
import torch

In [20]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Preparação dos dados

In [3]:
DATA_DIR = '/content/drive/MyDrive/tcc'

In [4]:
with open(f"{DATA_DIR}/vehicle_repair_and_maintenance_qa.json", 'r', encoding='utf-8') as fp:
    data = json.load(fp)

In [5]:
questions = [entry['data']['question'] for entry in data]
answers = [entry['data']['answer'] for entry in data]

In [6]:
test_size = 0.1
random_state = 42

In [7]:
dataset = Dataset.from_dict({'question': questions, 'answer': answers})
split_dataset = dataset.train_test_split(test_size=test_size, shuffle=True, seed=random_state)

train_dataset = split_dataset['train']
test_dataset = split_dataset['test']

In [8]:
train_dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 2408
})

In [9]:
test_dataset

Dataset({
    features: ['question', 'answer'],
    num_rows: 268
})

## Definição de métrica

In [18]:
metric = evaluate.load("rouge")
def calculate_rogue(predictions: list, labels: list) -> dict:
  decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip(), language='portuguese')) for pred in predictions]
  decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip(), language='portuguese')) for label in labels]
  return metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

## Treinamento do modelo

In [12]:
model_name = 'unicamp-dl/ptt5-base-portuguese-vocab'
max_length = 512
learning_rate = 3e-4
weight_decay = 0.01
n_epochs = 20
train_batch_size = 8
test_batch_size = 4

In [39]:
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/756k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/456 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [41]:
def tokenize_data(examples):
    model_inputs = tokenizer(examples['question'], max_length=max_length, truncation=True)
    labels = tokenizer(text_target=examples['answer'], max_length=max_length, truncation=True)
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

def compute_metrics(eval_predictions: EvalPrediction):
  preds, labels = eval_predictions
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  preds = np.where(preds != -100, preds, tokenizer.pad_token_id)
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  return calculate_rogue(decoded_preds, decoded_labels)

In [42]:
train_dataset_tokenized = train_dataset.map(tokenize_data, batched=True)
test_dataset_tokenized = test_dataset.map(tokenize_data, batched=True)

training_args = Seq2SeqTrainingArguments(
    output_dir=DATA_DIR,
    num_train_epochs=n_epochs,
    per_device_train_batch_size=train_batch_size,
    per_device_eval_batch_size=test_batch_size,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    predict_with_generate=True,
    generation_max_length=100
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_tokenized,
    eval_dataset=test_dataset_tokenized,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()

Map:   0%|          | 0/2408 [00:00<?, ? examples/s]

Map:   0%|          | 0/268 [00:00<?, ? examples/s]

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,2.151533,0.309679,0.152000,0.263155,0.269017
2,2.520300,1.702314,0.321032,0.161878,0.273037,0.278331
3,2.520300,1.431166,0.340660,0.179867,0.293359,0.298767
4,1.564200,1.285230,0.371950,0.214419,0.322485,0.327368
5,1.065500,1.190855,0.402525,0.247940,0.356317,0.362332
6,1.065500,1.131457,0.405154,0.249272,0.356133,0.360534
7,0.763700,1.110376,0.419852,0.271932,0.368405,0.374490
8,0.763700,1.121705,0.463612,0.325655,0.416926,0.422856
9,0.585300,1.109806,0.445263,0.301627,0.399239,0.403363
10,0.449200,1.102300,0.461109,0.322312,0.412775,0.418031


TrainOutput(global_step=6020, training_loss=0.698897139178558, metrics={'train_runtime': 4698.5158, 'train_samples_per_second': 10.25, 'train_steps_per_second': 1.281, 'total_flos': 1252578806046720.0, 'train_loss': 0.698897139178558, 'epoch': 20.0})

In [10]:
final_model_dir = f'{DATA_DIR}/final_model'

In [43]:
trainer.save_model(final_model_dir)

## Avaliação do modelo

In [37]:
del model
del tokenizer
torch.cuda.empty_cache()
gc.collect()

76

In [11]:
tokenizer = T5Tokenizer.from_pretrained(final_model_dir)
model = T5ForConditionalGeneration.from_pretrained(final_model_dir).to('cuda')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [12]:
def inference(model, tokenizer, questions):
  model.eval()

  inputs = tokenizer(questions, return_tensors="pt", padding=True).to('cuda')
  outputs = model.generate(**inputs, max_new_tokens=512)
  return tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [14]:
!pip install more-itertools
import more_itertools as mit
import pandas as pd

In [15]:
preds = []
batch_size = 100
for chunk in mit.chunked(questions, batch_size):
  preds.extend(inference(model, tokenizer, chunk))

In [16]:
assert(len(preds) == len(questions))

In [21]:
calculate_rogue(preds, answers)

{'rouge1': 0.8601157619982296,
 'rouge2': 0.8265815951918067,
 'rougeL': 0.849246836295395,
 'rougeLsum': 0.8516997281435753}

In [22]:
df = pd.DataFrame({'inputs': questions, 'preds': preds, 'labels': answers})

In [23]:
df.head()

,inputs,preds,labels
0,Quais foram as principais formas de transporte...,"Na origem da civilização, o homem utilizou pri...","Na origem da civilização, o homem utilizou pri..."
1,Quando foi construído o primeiro veículo com p...,"Em 1885, na Alemanha, Carl Benz construiu um v...",O primeiro triciclo a vapor foi construído em ...
2,Quais foram os problemas que a máquina a vapor...,A máquina a vapor apresentava baixo rendimento...,A máquina a vapor apresentava baixo rendimento...
3,Quem foi responsável pela invenção da máquina ...,A máquina endotérmica de quatro tempos movida ...,A máquina endotérmica de quatro tempos movida ...
4,Quem construiu o primeiro automóvel com motor ...,O primeiro automóvel com motor de quatro tempo...,O primeiro automóvel com motor de quatro tempo...


In [24]:
df.to_csv(f'{DATA_DIR}/train_result.csv')

In [25]:
print("Exact matches:", df[df['preds'] == df['labels']].shape[0])

Exact matches: 1763


## Publicação do modelo

In [26]:
from huggingface_hub import login
login()

In [28]:
model.save_pretrained(f"{DATA_DIR}/emgs/ptt5-qa")

In [29]:
model.push_to_hub("emgs/ptt5-qa")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/emgs/ptt5-qa/commit/d40b64c8a262a1d4b55b2064c6369f21a31ebfcf', commit_message='Upload T5ForConditionalGeneration', commit_description='', oid='d40b64c8a262a1d4b55b2064c6369f21a31ebfcf', pr_url=None, pr_revision=None, pr_num=None)

In [30]:
tokenizer.save_pretrained(f"{DATA_DIR}/emgs/ptt5-qa")

('/content/drive/MyDrive/tcc/emgs/ptt5-qa/tokenizer_config.json',
 '/content/drive/MyDrive/tcc/emgs/ptt5-qa/special_tokens_map.json',
 '/content/drive/MyDrive/tcc/emgs/ptt5-qa/spiece.model',
 '/content/drive/MyDrive/tcc/emgs/ptt5-qa/added_tokens.json')

In [31]:
tokenizer.push_to_hub("emgs/ptt5-qa")

CommitInfo(commit_url='https://huggingface.co/emgs/ptt5-qa/commit/330c5857fcd781adce938abfd0953d7480b43fa5', commit_message='Upload tokenizer', commit_description='', oid='330c5857fcd781adce938abfd0953d7480b43fa5', pr_url=None, pr_revision=None, pr_num=None)

## Explorando modelo

In [39]:
from huggingface_hub import login
login()

In [40]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

model = T5ForConditionalGeneration.from_pretrained('emgs/ptt5-qa').to("cuda")
tokenizer = T5Tokenizer.from_pretrained('emgs/ptt5-qa')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.6k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/756k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [46]:
inference(model, tokenizer, ["Qual é o material dos blocos de motores arrefecidos a água?"])

['Os blocos de ferro fundido são usados em motores arrefecidos a água.']

In [41]:
inference(model, tokenizer, ["Quais são algumas das características que a árvore de manivelas deve possuir?"])

['Deve ser feita de aços especiais que garantam uma resistência, de acordo com a potência do motor; a árvore de manivelas não deve ter cantos vivos onde possam aparecer trincas; deve ser maciça, pesada, daí ser confeccionada em uma peça inteiriça, fundida ou forjada; há um volante motor, acoplado à árvore de manivelas que compensa, com sua rotação, os tempos improdutivos do ciclo de trabalho de cada cilindro; deve ser balanceada, isto é, deve haver perfeita compensação de massas.']